In [1]:
import pandas as pd
import requests
from pandas.io.json import json_normalize
import json
import csv
import bar_chart_race as bcr
import matplotlib as plt 

In [2]:
pd.set_option("display.max_rows", None, "display.max_columns", None) # to see the complete DF and not the truncated version

In [3]:
league_url ='https://fantasy.premierleague.com/api/leagues-classic/1771282/standings/'
r = requests.get(league_url).text
json_data = json.loads(r)
# print (json.dumps(json_data, indent=2))

In [4]:
#converting to DF

In [5]:
league_data = pd.DataFrame(json_data['standings']['results'])

In [6]:
#creating list of dataframes for each individual player 

In [43]:
all_dataframes = []
player_id = league_data['entry'].tolist()
all_name = iter(league_data["player_name"].tolist())
no_gw = 9 # total no. of GW for the league

for team in player_id:

    team_url = 'https://fantasy.premierleague.com/api/entry/' + str(team) + '/history/' #inidividual address for each player
    p_id = requests.get(team_url).text
    teams_json_data = json.loads(p_id)
#     print(json.dumps(teams_json_data, indent =2))  #prints JSON for every player

    dataframe = pd.DataFrame(teams_json_data['current'], columns = ["total_points"])  #create dataframes for each player       
#     dataframe = pd.DataFrame(row, columns = ["GW", "total_points"])

    # addressing for player with old accounts 
    if len(dataframe['total_points']) >= 20:
        dataframe = dataframe.diff()
        dataframe = dataframe.iloc[-9:] #update this every GW
        dataframe = dataframe.cumsum()
        dataframe['total_points'].astype(int)
        
        
                
    dataframe = dataframe.rename(columns = {"total_points": next(all_name)})
    dataframe = dataframe.reset_index(drop=True)
    all_dataframes.append(dataframe)

In [44]:
final_dfs = pd.concat(all_dataframes, axis=1)

In [45]:
# Add GW columns here
GW = ["30+","31+","32+", "33+","34+","35+","36+",'37+','38+'] #update this every GW
final_dfs.insert (0, "GW", GW)

In [46]:
final_dfs = final_dfs.set_index("GW")

In [47]:
final_dfs

,Mubtasim Fuad,shareque zaman,Saraf Hasan,Mahbube Rabbani,Tasnimul Hasan,Pial Rahman,Likhon Amin,Tauhid Sefo,Mahedur Rahman,Abrar Hossain,Muntasir Azad,Asif Bin Zaman,Zabir Sami
GW,,,,,,,,,,,,,
30+,69,95,72,61,59,92,64.0,66.0,77.0,56,55.0,61,73
31+,137,164,135,124,114,148,113.0,140.0,151.0,125,121.0,113,148
32+,230,260,220,187,199,209,190.0,196.0,204.0,223,176.0,162,184
33+,286,311,288,259,259,259,245.0,267.0,246.0,277,233.0,228,240
34+,347,371,346,357,343,329,318.0,336.0,304.0,327,289.0,302,291
35+,400,403,397,409,395,394,372.0,378.0,359.0,358,351.0,347,334
36+,451,454,438,466,441,444,414.0,440.0,397.0,397,396.0,388,381
37+,513,510,510,520,486,498,476.0,480.0,455.0,444,445.0,445,454
38+,597,578,576,573,570,551,545.0,529.0,528.0,504,504.0,498,491


In [12]:
# Visuualization

In [49]:

bcr.bar_chart_race(
    df=final_dfs,
    filename= None,
    orientation='h',
    sort='desc',
    n_bars=13,
    fixed_order=False,
    fixed_max=True,
    steps_per_period=85,
    interpolate_period=False,
    label_bars=True,
    bar_size=.95,
    period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center'},
    period_fmt='GW {x}',
    period_summary_func=lambda v, r: {'x': .99, 'y': .38,
                                      's': f'Highest point in GW: {v.nlargest(13).mean():,.0f}',
                                      'ha': 'right', 'size': 8, 'family': 'Courier New'},
    perpendicular_bar_func='median',
    period_length=6500,
    figsize=(5, 3),
    dpi=144,
    cmap='dark12',
    title='Nuno Nothing Jon Snow Dancing Chair',
    title_size='',
    bar_label_size=7,
    tick_label_size=7,
    shared_fontdict={'family' : 'Helvetica', 'color' : '.1'},
    scale='linear',
    writer=None,
    fig=None,
    bar_kwargs={'alpha': .7},
    filter_column_colors=False)  

Exception: You do not have ffmpeg installed on your machine. Download
                            ffmpeg from here: https://www.ffmpeg.org/download.html.
                            
                            Matplotlib's original error message below:

                            The dictionary returned from `<lambda>` must contain "x", "y", and "s"
                            